# 0. 패키지 임포트

In [1]:
from dotenv import load_dotenv
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_upstage import UpstageEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from decouple import config
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\Admin\anaconda3\envs\llm-lab\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. 기본 세팅

In [16]:
load_dotenv()

True

In [17]:
pc = Pinecone()

In [18]:
# embedding = OpenAIEmbeddings(model="text-embedding-3-large")
embedding = UpstageEmbeddings(model="solar-embedding-1-large")
index_name = config("PINECONE_INDEX")


In [19]:
database = PineconeVectorStore(
    embedding=embedding, 
    index_name=index_name
)

In [20]:
retriever = database.as_retriever(search_kwargs={'score_threshold': 0.8,"k": 3})
prompt = hub.pull('rlm/rag-prompt')
llm = ChatOpenAI(model="gpt-4.1-nano")

In [21]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = retriever,
    chain_type_kwargs={"prompt": prompt},
)

# 2. 선다형 문제로 검증 실행

In [22]:
df = pd.read_csv("법령문제_금융은행보험.csv")
df.sample()

,법령명,문제유형,난이도,문제내용,보기1,보기2,보기3,보기4,정답,해설
1151,화재로 인한 재해보상과 보험가입에 관한 법률(법률)(제19265호)(20230922),사지선다형,중,한국화재보험협회의 설립에 대한 규정은 어디에 있는가?,A. 제10조,B. 제11조,C. 제12조,D. 제13조,2,제11조에 따르면 손해보험회사는 한국화재보험협회를 설립해야 한다.


In [23]:
success, fail = 0, 0
gpt = []
act = []
for i in range(50):
    sample = df.loc[i]
    question = """아래 문제를 보고 답을 고르시오.
만약 보기 중 3번째 보기가 정답이라면 3 이라고 출력하시오.
보기가 A,B,C,D 라면 1,2,3,4 라고 생각하고 답변하시오.
오직 숫자만 출력하시오.
출력 예시: 3
"""
    question += sample['문제내용'] + '\n'

    for i in range(1,5):
        col = '보기' + str(i)
        question += sample[col] + '\n'

    message = qa_chain.invoke({"query":question})['result']
    if message == str(sample['정답']):
        success += 1
    else:
        fail += 1
    gpt.append(message)
    act.append( str(sample['정답']))

In [25]:
print('정답 갯수 :', success)
print('오답 갯수 :', fail)
print('정확도 :', (success)/(success+fail))
print('챗봇에 대답: ', gpt)
print('실제 답    : ', act)

정답 갯수 : 29
오답 갯수 : 21
정확도 : 0.58
챗봇에 대답:  ['3', '2', '2', '2', '3', '2', '3', '2', '4', '2', '4', '2', '2', '3', '4', '3', '1', '2', '3', '2', '2', '2', '2', '2', '2', '2', '3', '4', '3', '2', '2', '2', '3', '3', '2', '2', '1', '3', '3', '2', '2', '3', '2', '3', '2', '2', '2', '3', '3', '6']
실제 답    :  ['1', '3', '2', '3', '3', '4', '4', '1', '2', '2', '3', '2', '2', '2', '3', '2', '1', '2', '3', '2', '2', '2', '2', '1', '2', '2', '3', '1', '1', '2', '2', '2', '1', '2', '2', '2', '1', '3', '1', '2', '2', '3', '2', '1', '4', '2', '2', '3', '1', '3']


# 3. 단답형 문제로 검증 실행

In [8]:
df = pd.read_csv("단답형_문제.csv")
df.sample()

,법령명,문제유형,난이도,문제내용,정답,해설
618,한국은행법 시행령(대통령령)(제35228호)(20250131),단답형,상,한국은행의 설립등기 시 첨부해야 하는 서류는 무엇인가?,"정관, 정관인가서의 사본",제2조에 명시된 바와 같이 정관과 정관인가서의 사본을 첨부해야 함.


In [9]:
results = []
SIMILARITY_THRESHOLD = 0.7

In [14]:
success, fail = 0, 0
gpt = []
act = []
for i in range(50):

    question = """아래 문제를 보고 짧고 간단하게 답을 단답식으로 제출하시오.
    좋은 답변들 예시 : ['대주주와의 거래', '지배구조법 대주주', '금융위원회', '제12조', '5%']
    이와같이 30자 이내로 핵심 단어의 단답형으로 답변하시오.
    """

    sample = df.loc[i]
    question += sample['문제내용']
    answer = sample['정답']


    
    message = qa_chain.invoke({"query": question})['result']

    ground_truth_embedding = embedding.embed_query(answer)
    prediction_embedding = embedding.embed_query(message)
    
    score = cosine_similarity(
        np.array(ground_truth_embedding).reshape(1, -1),
        np.array(prediction_embedding).reshape(1, -1)
    )[0][0]
    
    if score >= SIMILARITY_THRESHOLD:
        success += 1
    else:
        fail += 1
        
    gpt.append(message)
    act.append(answer)

In [15]:
print('정답 갯수 :', success)
print('오답 갯수 :', fail)
print('정확도 :', (success)/(success+fail))
print('챗봇에 대답: ', gpt)
print('실제 답    : ', act)

정답 갯수 : 17
오답 갯수 : 33
정확도 : 0.34
챗봇에 대답:  ['개인금융채권', '개인금융채권 연체 사유', "법 제12조제1항 '대통령령으로 정하는 기간'은 5년이다.", '금액: 10억', '추심 위탁 통지 방법', '개인금융채권은 지급보증성 자금지원 증권.', '금융회사란 은행, 보험사, 증권사 등 금융업을 하는 법인.', '채무상환 내용 변경', '추심의 정의는 채권회수 또는 강제집행.', '퇴임일 또는 퇴직일', '채권추심자란 개인금융채권을 추심하는 자', '이자제한법 제14조', '개인금융채권 양도제한', '별표 1~17', '한국은행', '금융위원회', '2024년 12월 3일', '기획재정부', '국제금융기구 출자금 반영; 예산; 정부증권', '모든 답변이 내게 제공된 자료와 관련이 없으므로, 정확한 답을 알 수 없다.', '국제금융기구가입법', '국회승인', '금융거래지표법 시행령', '산출업무규정 첨부서류', '5명 이상', '중단 시 안내사항 없음', '2020년 11월 27일', '법 제6조제4항', '금융거래지표법', '기초자료는 거래 기록 자료', '2019년', '중요지표 산출업무규정 검토', '7월 31일', '위험관리기준', '금융시장 안정과 금융소비자 보호', '대통령령으로 정하는 금융업 또는 회사', '2021년 6월 30일', '금융복합기업집단법', '지정 해제 사유 없음', '내부거래란 대주주 또는 특수관계자와의 자금, 증권, 자산, 용역 거래행위.', '대주주=금융회사의 지배구조법 제2조제6호', '금융위원회', '금융위험관리법, 제12조, 제16조', '5%', '금융기관의 주요 출자자란: 특정 지분보유자', '자금지원 규모 협의', '대주주 및 주요출자자', '자금지원 규모, 경영정상화계획, 협의', '금융기관은 금융업무를 수행하는 회사 또는 기관이다.', '부채초과,경영악화,회생불가']
실제 답    :  ['법 제3조제1항 각 호의 권리로 담보된 개인금융채권', '연체기간이나 연체횟수를 초과한 경우'

In [26]:
test_question = "어선원 재해보상보험의 손실보전준비금은 어떤 용도로 사용되는가?"
retrieved_docs_with_scores = database.similarity_search_with_score(test_question, k=5)
print(f"질문: \"{test_question}\"\n")
print("--- 검색된 문서 목록 (유사도 점수 높은 순) ---\n")
for i, (doc, score) in enumerate(retrieved_docs_with_scores):
    print(f"[문서 {i+1} / 유사도 점수: {score:.4f}]")
    print(f"출처: {doc.metadata.get('source', 'N/A')}") # 문서의 출처 파일명
    print("-" * 20)
    print(doc.page_content) # 검색된 텍스트 조각의 내용
    print("\n" + "="*50 + "\n")

질문: "어선원 재해보상보험의 손실보전준비금은 어떤 용도로 사용되는가?"

--- 검색된 문서 목록 (유사도 점수 높은 순) ---

[문서 1 / 유사도 점수: 0.5591]
출처: ..\..\내파일\보험법령\어선원 및 어선 재해보상보험법(법률)(제20132호)(20240724).docx
--------------------
어선원 및 어선 재해보상보험법

어선원 및 어선 재해보상보험법 ( 약칭: 어선원재해보험법 )

[시행 2024. 7. 24.] [법률 제20132호, 2024. 1. 23., 일부개정]

해양수산부(소득복지과) 044-200-5468, 5471



제1장 총칙 <개정 2009. 5. 27.>



제1조(목적) 이 법은 어업에 종사하는 어선원 등과 어선에 대한 재해보상보험사업을 시행하여 어선원 등의 재해를 신속ㆍ공정하게 보상하여 재활 및 사회복귀를 지원하고 재해를 입은 어선의 복구를 촉진함으로써 어선원 등을 보호하고, 어업경영의 안정에 이바지함을 목적으로 한다. <개정 2021. 6. 15.>

[전문개정 2009. 5. 27.]



제2조(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다.

1. “어선”이란 「어선법」 제2조제1호라목에 따른 선박을 말한다.

2. “어선원”이란 임금을 받을 목적으로 어선에서 근로를 제공하기 위하여 고용된 사람을 말한다.

3. “가족어선원”이란 어선의 소유자의 배우자(사실혼 관계에 있는 사람을 포함한다) 및 직계 존속ㆍ비속으로서 어선에서 근로를 제공하는 사람을 말한다.

4. “어선의 소유자”란 선주, 어선차용인, 어선관리인, 용선인 등 명칭에 상관 없이 어선원을 고용하고 그 어선원에게 임금을 지급하거나 자기가 직접 또는 가족어선원과 함께 어업 활동을 하는 자를 말한다.

5. “임금”, “통상임금”, “승선평균임금”이란 「선원법」에 따른 각각의 임금, 통상임금, 승선평균임금을 말한다.

6. “어선원등의 재해”란 어선원, 가족어선원 및 어선의 소유자(이하 “어선원등”이라 한다)가 어업 활동